In [9]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

class IndianInsurancePredictor:
    def __init__(self):
        self.model = None
        self.encoders = {}
        self.regions = ['North', 'South', 'East', 'West', 'Central']
        self._train_model()
    
    def _train_model(self):
        """Train Indian insurance model."""
        np.random.seed(42)
        n_samples = 5000
        
        data = {
            'age': np.random.randint(18, 70, n_samples),
            'gender': np.random.choice(['Male', 'Female'], n_samples),
            'bmi': np.random.normal(25.5, 5.5, n_samples).clip(15, 45),
            'family_size': np.random.choice([1, 2, 3, 4, 5, 6], n_samples),
            'smoker': np.random.choice(['Yes', 'No'], n_samples, p=[0.15, 0.85]),
            'region': np.random.choice(self.regions, n_samples),
            'diabetes': np.random.choice(['Yes', 'No'], n_samples, p=[0.12, 0.88]),
            'hypertension': np.random.choice(['Yes', 'No'], n_samples, p=[0.18, 0.82]),
            'asthma': np.random.choice(['Yes', 'No'], n_samples, p=[0.08, 0.92]),
            'heart_disease': np.random.choice(['Yes', 'No'], n_samples, p=[0.04, 0.96]),
        }
        
        df = pd.DataFrame(data)
        
        # Indian insurance premiums (₹5K-₹5L)
        df['premium'] = (
            8000 + df['age'] * 150 + (df['bmi'] - 22) * 300 +
            (df['family_size'] - 1) * 2500 +
            (df['smoker'] == 'Yes') * 35000 +
            (df['diabetes'] == 'Yes') * 25000 +
            (df['hypertension'] == 'Yes') * 20000 +
            (df['asthma'] == 'Yes') * 15000 +
            (df['heart_disease'] == 'Yes') * 75000 +
            np.random.normal(0, 8000, n_samples)
        ).clip(5000, 500000)
        
        # Encode categoricals ✅ FIXED
        categorical_cols = ['gender', 'smoker', 'region', 'diabetes', 'hypertension', 'asthma', 'heart_disease']
        for col in categorical_cols:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
            self.encoders[col] = le
        
        X = df.drop('premium', axis=1)
        y = df['premium']
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        self.model = LinearRegression()
        self.model.fit(X_train, y_train)
        
        score = self.model.score(X_test, y_test)
        print(f"✅ Model trained successfully! Test R² = {score:.3f}")
    
    def predict_premium(self, age, gender, bmi, family_size, smoker, region, 
                       diabetes, hypertension, asthma, heart_disease):
        """Predict premium with error handling."""
        try:
            features = np.array([[
                age,
                self.encoders['gender'].transform([gender])[0],
                bmi,
                family_size,
                self.encoders['smoker'].transform([smoker])[0],
                self.encoders['region'].transform([region])[0],
                self.encoders['diabetes'].transform([diabetes])[0],
                self.encoders['hypertension'].transform([hypertension])[0],
                self.encoders['asthma'].transform([asthma])[0],
                self.encoders['heart_disease'].transform([heart_disease])[0]
            ]])
            
            premium = self.model.predict(features)[0]
            premium = max(5000, min(500000, round(premium / 100) * 100))
            
            risk_score = self._calculate_risk(age, bmi, smoker, diabetes, hypertension, asthma, heart_disease)
            risk_level, advice = self._get_advice(risk_score, bmi, smoker, diabetes, hypertension, asthma, heart_disease, age)
            eligibility = "✅ POLICY ISSUABLE" if risk_score < 80 else "⚠️ UNDERWRITING NEEDED"
            
            return premium, risk_level, advice, eligibility
        except Exception as e:
            return 0, "ERROR", "Prediction failed", "ERROR"
    
    def _calculate_risk(self, age, bmi, smoker, diabetes, hypertension, asthma, heart_disease):
        risk = min(70, age * 1.2)
        if bmi < 23: risk += 15
        elif bmi < 27.5: risk += 45
        else: risk += 70
        if smoker == 'Yes': risk += 70
        if diabetes == 'Yes': risk += 55
        if hypertension == 'Yes': risk += 45
        if asthma == 'Yes': risk += 35
        if heart_disease == 'Yes': risk += 90
        return min(100, risk)
    
    def _get_advice(self, risk_score, bmi, smoker, diabetes, hypertension, asthma, heart_disease, age):
        bmi_cat = "Normal" if 18.5 <= bmi < 23 else "High Risk"
        conditions = [c for c, s in [('Diabetes', diabetes), ('BP', hypertension), 
                                    ('Asthma', asthma), ('Heart', heart_disease)] if s == 'Yes']
        
        if risk_score < 30:
            return "🟢 LOW RISK", "✅ Excellent profile. No waiting period. Maintain lifestyle."
        elif risk_score < 60:
            return "🟡 MODERATE RISK", f"⚠️ Moderate risks. Annual checkups recommended.\nConditions: {', '.join(conditions) if conditions else 'None'}"
        else:
            return "🔴 HIGH RISK", (
                f"🚨 Critical profile. 4-year waiting period likely.\n"
                f"• Immediate specialist consultation needed\n"
                f"• Loading: 50-200% expected"
            )

class IndianInsuranceGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("🇮🇳 AI Health Insurance Calculator")
        self.root.geometry("950x850")
        
        self.predictor = IndianInsurancePredictor()
        self._build_ui()
    
    def _build_ui(self):
        style = ttk.Style()
        style.theme_use('clam')
        
        # Header
        header = ttk.Frame(self.root, padding="25")
        header.pack(fill=tk.X)
        ttk.Label(header, text="🇮🇳 AI Health Insurance Premium Predictor", 
                 font=('Segoe UI', 20, 'bold')).pack()
        ttk.Label(header, text="IRDAI Compliant | Indian Medical Conditions", 
                 font=('Segoe UI', 12)).pack(pady=(0, 20))
        
        # Form
        form_frame = ttk.LabelFrame(self.root, text="📋 Medical Profile", padding="20")
        form_frame.pack(fill=tk.X, padx=25, pady=(0, 20))
        
        # 3 Columns ✅ FIXED LAYOUT
        self.fields = {}
        col1 = ttk.Frame(form_frame)
        col1.grid(row=0, column=0, padx=(0, 15))
        
        col2 = ttk.Frame(form_frame)
        col2.grid(row=0, column=1, padx=(0, 15))
        
        col3 = ttk.Frame(form_frame)
        col3.grid(row=0, column=2)
        
        # Col 1: Numbers
        self._add_spinbox(col1, "Age", "age", 18, 70, 30)
        self._add_spinbox(col1, "BMI", "bmi", 15.0, 45.0, 23.0, 0.1)
        self._add_spinbox(col1, "Family Size", "family_size", 1, 6, 2)
        
        # Col 2: Basic
        self._add_dropdown(col2, "Gender", "gender", ['Male', 'Female'], 'Male')
        self._add_dropdown(col2, "Smoker", "smoker", ['Yes', 'No'], 'No')
        self._add_dropdown(col2, "Region", "region", self.predictor.regions, 'South')
        
        # Col 3: Medical ✅ FIXED KEY NAMES
        self._add_dropdown(col3, "Diabetes", "diabetes", ['Yes', 'No'], 'No')
        self._add_dropdown(col3, "Hypertension", "hypertension", ['Yes', 'No'], 'No')
        self._add_dropdown(col3, "Asthma", "asthma", ['Yes', 'No'], 'No')
        self._add_dropdown(col3, "Heart Disease", "heart_disease", ['Yes', 'No'], 'No')
        
        # Buttons
        btn_frame = ttk.Frame(form_frame)
        btn_frame.grid(row=1, column=0, columnspan=3, pady=20)
        ttk.Button(btn_frame, text="🔮 Calculate Premium", command=self.predict).pack(side=tk.LEFT, padx=10)
        ttk.Button(btn_frame, text="🔄 Reset", command=self.clear).pack(side=tk.LEFT)
        
        # Results
        result_frame = ttk.LabelFrame(self.root, text="📊 Results", padding="20")
        result_frame.pack(fill=tk.BOTH, expand=True, padx=25, pady=(0, 25))
        
        self.premium_label = ttk.Label(result_frame, text="Premium: ₹ --", 
                                     font=('Segoe UI', 24, 'bold'), foreground='#1B5E20')
        self.premium_label.pack(pady=20)
        
        status_frame = ttk.Frame(result_frame)
        status_frame.pack(fill=tk.X, pady=(0, 20))
        self.eligibility_label = ttk.Label(status_frame, text="Status: --", font=('Segoe UI', 13, 'bold'))
        self.eligibility_label.pack(side=tk.LEFT)
        self.risk_label = ttk.Label(status_frame, text="Risk: --", font=('Segoe UI', 13, 'bold'))
        self.risk_label.pack(side=tk.RIGHT)
        
        self.advice_text = tk.Text(result_frame, height=10, font=('Segoe UI', 10), state=tk.DISABLED)
        self.advice_text.pack(fill=tk.BOTH, expand=True)
    
    def _add_spinbox(self, parent, label, key, min_val, max_val, default, increment=1):
        ttk.Label(parent, text=label, font=('Segoe UI', 11, 'bold')).pack(anchor=tk.W)
        spin = ttk.Spinbox(parent, from_=min_val, to=max_val, increment=increment, width=10)
        spin.set(default)
        spin.pack(pady=(0, 10))
        self.fields[key] = spin
    
    def _add_dropdown(self, parent, label, key, values, default):
        ttk.Label(parent, text=label, font=('Segoe UI', 11, 'bold')).pack(anchor=tk.W)
        combo = ttk.Combobox(parent, values=values, state='readonly', width=9)
        combo.set(default)
        combo.pack(pady=(0, 10))
        self.fields[key] = combo
    
    def predict(self):
        try:
            # Get values ✅ FIXED KEY ACCESS
            inputs = {
                'age': int(self.fields['age'].get()),
                'gender': self.fields['gender'].get(),
                'bmi': float(self.fields['bmi'].get()),
                'family_size': int(self.fields['family_size'].get()),
                'smoker': self.fields['smoker'].get(),
                'region': self.fields['region'].get(),
                'diabetes': self.fields['diabetes'].get(),
                'hypertension': self.fields['hypertension'].get(),
                'asthma': self.fields['asthma'].get(),
                'heart_disease': self.fields['heart_disease'].get()
            }
            
            premium, risk_level, advice, eligibility = self.predictor.predict_premium(**inputs)
            
            self.premium_label.config(text=f"💰 Annual Premium: ₹ {premium:,}")
            self.eligibility_label.config(text=f"📜 {eligibility}", foreground='green' if 'ISSUABLE' in eligibility else 'red')
            self.risk_label.config(text=f"⚠️ {risk_level}", foreground='green' if 'LOW' in risk_level else 'red')
            
            self.advice_text.config(state=tk.NORMAL)
            self.advice_text.delete(1.0, tk.END)
            self.advice_text.insert(tk.END, advice)
            self.advice_text.config(state=tk.DISABLED)
            
        except Exception as e:
            messagebox.showerror("Error", f"Prediction failed: {str(e)}")
    
    def clear(self):
        defaults = {
            'age': '30', 'bmi': '23.0', 'family_size': '2',
            'gender': 'Male', 'smoker': 'No', 'region': 'South',
            'diabetes': 'No', 'hypertension': 'No', 'asthma': 'No', 'heart_disease': 'No'
        }
        for key, value in defaults.items():
            self.fields[key].set(value)
        
        self.premium_label.config(text="Premium: ₹ --")
        self.eligibility_label.config(text="Status: --")
        self.risk_label.config(text="Risk: --")
        self.advice_text.config(state=tk.NORMAL)
        self.advice_text.delete(1.0, tk.END)
        self.advice_text.config(state=tk.DISABLED)

if __name__ == "__main__":
    root = tk.Tk()
    app = IndianInsuranceGUI(root)
    root.mainloop()


✅ Model trained successfully! Test R² = 0.893
